In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read in Data


* Must have the 'Ridership Model Data.csv' in the project folder;
* Create model_data for training the model which has non-relevant features dropped; and
* Create features to be used for generating future feature values.



In [ ]:
# Read in data and split in to training and prediction sets

df = pd.read_csv('Ridership Model Data.csv')
model_data = df.drop(['ISOWeek','Month','Start of Week','Population Growth Rate',
                      'critical cases','BC Vaccination Rate'], axis=1)
features = model_data.drop(['Total Boardings'], axis=1)
feat_desc = features.describe()

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def get_preds(year,restaurant_bookings,gas_price,uni_season,emp,wfh,covid=False):
  '''
  Generate prediction for specified values.
  '''
  if year > 2021:
    inc = year - 2021
  else:
    inc = 0

  if covid:
    yr = [2021] * 100
  else:
    yr = [2019] * 100
  
  revenue_hours = np.linspace(min(features['Revenue Hours']),max(features['Revenue Hours']),100)
  restaurant_bookings = [restaurant_bookings] * 100
  gas_price = [gas_price] * 100
  uni_season = [uni_season] * 100
  employment = [feat_desc['Employment']['mean'] * (1 + inc * emp)] * 100
  wfh = [feat_desc['WFH']['mean'] * (1 + inc * wfh)] * 100
  if covid:
    hospitalizations = [feat_desc['hospitalizations']['mean']] * 100
  else:
    hospitalizations = [0] * 100
  
  test_data = np.array([yr, revenue_hours, restaurant_bookings, gas_price, uni_season, employment, wfh, hospitalizations]).transpose()
  test_data = pd.DataFrame(test_data,columns=features.columns)
  test_data = scaler.transform(test_data)
  prediction = model.predict(test_data)

  coefs = np.polyfit(revenue_hours, prediction,1)

  print('An increase of 1 Rev. Hr. creates an increase of ' + str(coefs[0][0]) + ' weekly boardings.')
  print('\n')

# Train Test Split

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Scale the data for model training.
X = model_data.drop('Total Boardings',axis=1)
y = model_data['Total Boardings']

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Create and Train the Model

* Hyperparameters have been selected based on previous testing of the model.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# Deep fully-connected neural network consisting of four layers (8 -> 5 -> 3 -> 1).
# Rectified linear unit activation functions for non-ouput layers.
# No dropouts as this hindered performance in the model testing.
# Adam optimizer and mse loss function for regression on a single output variable.
model = Sequential()

model.add(Dense(8,activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(3,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='Adam', loss='mse')

In [ ]:
# Fit the model to all our data - 200 epochs chosen based on test runs with validation data.
model.fit(x=X,y=y,batch_size=1,epochs=200)

Epoch 1/200
157/157 [==============================] - 1s 931us/step - loss: 210359255040.0000
Epoch 2/200
157/157 [==============================] - 0s 939us/step - loss: 210357764096.0000
Epoch 3/200
157/157 [==============================] - 0s 1ms/step - loss: 210349588480.0000
Epoch 4/200
157/157 [==============================] - 0s 1ms/step - loss: 210313789440.0000
Epoch 5/200
157/157 [==============================] - 0s 1ms/step - loss: 210217467904.0000
Epoch 6/200
157/157 [==============================] - 0s 890us/step - loss: 210024464384.0000
Epoch 7/200
157/157 [==============================] - 0s 934us/step - loss: 209694867456.0000
Epoch 8/200
157/157 [==============================] - 0s 1ms/step - loss: 209189502976.0000
Epoch 9/200
157/157 [==============================] - 0s 1ms/step - loss: 208466280448.0000
Epoch 10/200
157/157 [==============================] - 0s 968us/step - loss: 207484010496.0000
Epoch 11/200
157/157 [==============================] - 0s 

In [ ]:
interact(get_preds,year=[('2022',2022),('2023',2023),('2024',2024)],
         restaurant_bookings=widgets.FloatSlider(value=0,min=-1.0,max=1.0,step=.1,description='restaurant bookings'),
         gas_price=widgets.IntSlider(value=feat_desc['Gas Price (C/L)']['mean'],
                                     min=feat_desc['Gas Price (C/L)']['mean'] - 3 * feat_desc['Gas Price (C/L)']['std'],
                                     max=feat_desc['Gas Price (C/L)']['mean'] + 3 * feat_desc['Gas Price (C/L)']['std'],
                                     step=1, description='gas price'),
         uni_season=widgets.Dropdown(options=[('In Session',1),('Not in Session',2),('Holiday',3)],description='university'),
         emp=widgets.FloatSlider(value=0.0,
                               min=-.05,
                               max=.05,
                               step=.01, description='employment % change per year'),
         wfh=widgets.FloatSlider(value=0.0,
                               min=-1.0,
                               max=1.0,
                               step=.1, description='wfh % change per year')
         )

interactive(children=(Dropdown(description='year', options=(('2022', 2022), ('2023', 2023), ('2024', 2024)), v…

<function __main__.get_preds>

In [ ]:
21000/ df['Total Boardings'].describe()['mean']

0.04954222801001267

In [ ]:
20000/884000

0.02262443438914027